In [ ]:
## FUNCTION 2
def get_acquisitions_data(ehraid, hops=0, save=False, df=False):


    '''

    Takes in an ehraid (string) and returns a list of dictionaries with information about the firm and all the firms it has taken over. 
    This function finds both direct and indirect acquisitions. For example, if firm A has taken over firm B, and firm B has taken over firm C, 
    the function will return information about firm A, firm B, and firm C. Starting from A, the function will count the number of "hops" to reach
    the last firm in the chain. The function has an option to save the data to a JSON file.
    
    Parameters:
    ehraid (str): ehraid of the firm to start from
    hops (int): DO NOT CHANGE THIS VALUE, it is used as a counter for the number of hops from the initial ehraid
    save (bool): if True, saves the data to a JSON file called 'takeovers.json'
    df (bool): if True, returns the data as a pandas DataFrame

    Returns:

    if df=False (default):
    acquisitions (list): a list of dictionaries, each containing information about a firm and the firms it has taken over

    if df=True:
    acquisitions (DataFrame): a pandas DataFrame containing information about all firms and the firms they have taken over
    
    '''
    

    acquisitions = []

    # check if ehraid is a dictionary: if it is, get it as a string
    ehraid = ehraid.get('ehraid') if isinstance(ehraid, dict) else ehraid

    # get the data of the firm with the given ehraid from the API 
    url = f"https://www.zefix.ch/ZefixREST/api/v1/firm/{ehraid}"
    r = requests.get(url)
    data = r.json()

    # check if the request was successful, return error message if not
    if r.status_code == 400:
        return "400 Bad Request Error"
    elif r.status_code == 401:
        return "401 Unauthorized Error"
    elif r.status_code == 403:
        return "403 Forbidden Error"
    elif r.status_code == 404:
        return "404 Not Found Error"
    elif r.status_code == 408:
        return "408 Request Timeout Error"
    elif r.status_code == 500:
        return "500 Internal Server Error"
    elif r.status_code == 502:
        return "502 Bad Gateway Error"
    elif r.status_code == 504:
        return "504 Gateway Timeout Error"
    elif r.status_code != 200:
        return "Unexpected Error, the request was not successful"
    
    # Extract data of firm as a dictionary, with the number of hops from the initial ehraid
    firm = {
        "name": data.get("name"),
        "ehraid": data.get("ehraid"),
        "legalSeat": data.get("legalSeat"),
        "legalFormId": data.get("legalFormId"),
        "status": data.get("status"),
        "cantonalExcerptWeb": data.get("cantonalExcerptWeb"),
        "deleteDate": data.get("deleteDate"),
        "hops": hops
    }
    
    # translate the information in 'status' to english
    if firm["status"]:
        if firm['status'].upper() == 'EXISTIEREND':
            firm['status'] = 'Active'
        elif firm['status'].upper() == 'GELOESCHT':
            firm['status'] = 'Deleted'
        elif firm['status'].upper() == 'IN_AUFLOESUNG':
            firm['status'] = 'In Liquidation'
        else:
            translator = Translator()
            status_english = translator.translate(firm['status'], dest='en')
            firm['status'] = status_english.text
        
    acquisitions.append(firm)

    # if hastakenover is not empty, for each element in hastakenover run again this function with the element as input 
    # (since it's the ehrid of the firms taken over) and increase hops by 1
    
    if data["hasTakenOver"] is not None:
        for element in data["hasTakenOver"]:
            acquisitions.extend(get_acquisitions_data(element, hops + 1))

        
    # Save data to JSON file if user 
    if save == True:
        with open('takeovers.json', 'w') as file:
            json.dump(acquisitions, file, indent=4)
        print("Data saved to takeovers.json")

    if df==True:
        acquisitions = pd.DataFrame(acquisitions)

    return acquisitions

In [ ]:
##EXAMPLE FUNCTION 2
get_acquisitions_data('1578596')
a = get_acquisitions_data('388851', save=False, df=False)
